In [2]:
import cv2
import numpy as np
from skimage.morphology import remove_small_objects, binary_closing, disk

def clean_mask(mask):
    mask = mask > 0

    # fill holes gently
    mask = cv2.morphologyEx(
        mask.astype(np.uint8),
        cv2.MORPH_CLOSE,
        np.ones((3,3), np.uint8),
        iterations=1
    )

    # remove only very tiny noise
    mask = remove_small_objects(mask, min_size=50)

    # smooth slightly
    mask = binary_closing(mask, disk(1))

    return (mask * 255).astype(np.uint8)


In [3]:
def keep_largest_component(mask):
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask)

    if num_labels <= 1:
        return mask

    largest = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
    cleaned = np.zeros_like(mask)
    cleaned[labels == largest] = 255
    return cleaned


In [4]:
from skimage.morphology import skeletonize

def get_centerline(mask):
    binary = mask > 0
    skeleton = skeletonize(binary)
    return skeleton


In [5]:
from scipy.ndimage import distance_transform_edt

def compute_diameter(mask, skeleton):
    binary = mask > 0

    dist = distance_transform_edt(binary)

    diameters = dist[skeleton] * 2  # radius → diameter
    return diameters


In [6]:
from scipy.ndimage import distance_transform_edt

def compute_diameter(mask, skeleton):
    binary = mask > 0

    dist = distance_transform_edt(binary)

    diameters = dist[skeleton] * 2  # radius → diameter
    return diameters


In [7]:
def compute_stenosis_percent(diameters):
    diameters = np.array(diameters)

    if len(diameters) < 20:
        return None, None, None

    # Remove extreme noise
    diameters = diameters[
        (diameters > np.percentile(diameters, 10)) &
        (diameters < np.percentile(diameters, 90))
    ]

    if len(diameters) < 10:
        return None, None, None

    D_min = np.min(diameters)
    D_ref = np.mean(np.sort(diameters)[-10:])  # average of widest section

    stenosis = (1 - D_min / D_ref) * 100
    return stenosis, D_min, D_ref


In [8]:
mask = cv2.imread(r"mmask.png", 0)

mask = clean_mask(mask)
mask = keep_largest_component(mask)

skeleton = get_centerline(mask)
diameters = compute_diameter(mask, skeleton)

stenosis, D_min, D_ref = compute_stenosis_percent(diameters)

print(f"Stenosis: {stenosis:.2f}%")


Stenosis: 40.00%


C:\Users\Goyam Jain\AppData\Local\Temp\ipykernel_51196\3620001085.py:17: FutureWarning: Parameter `min_size` is deprecated since version 0.26.0 and will be removed in 2.0.0 (or later). To avoid this warning, please use the parameter `max_size` instead. For more details, see the documentation of `remove_small_objects`. Note that the new threshold removes objects smaller than **or equal to** its value, while the previous parameter only removed smaller ones.
  mask = remove_small_objects(mask, min_size=50)
d:\Stenosis-detection\venv\Lib\site-packages\skimage\_shared\utils.py:386: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  return func(*args, **kwargs)
C:\Users\Goyam Jain\AppData\Local\Temp\ipykernel_51196\3620001085.py:20: FutureWarning: `binary_closing` is deprecated since version 0.26 and will be removed in version 0.28. Use `skimage.morphology.closing` instead.
  mask = binary_closing(mask, disk(1))


In [9]:
stenosis, D_min, D_ref = compute_stenosis_percent(diameters)

if stenosis is None:
    print("Stenosis not reliable for this image")
else:
    print(f"Stenosis: {stenosis:.2f}%")
    print(f"D_min: {D_min}, D_ref: {D_ref}")


Stenosis: 40.00%
D_min: 6.0, D_ref: 10.0
